In [2]:
#optional code if warnings become over bearing 
import warnings
warnings.filterwarnings('ignore')

In [3]:
import ipywidgets as widgets
from IPython.display import display

In [15]:
w = widgets.IntSlider(max=120, min=20, description='Temperature: ')
display(w)

IntSlider(value=20, description='Temperature: ', max=120, min=20)

In [9]:
print(w.value)

77


In [6]:
w.keys

['_dom_classes',
 '_model_module',
 '_model_module_version',
 '_model_name',
 '_view_count',
 '_view_module',
 '_view_module_version',
 '_view_name',
 'continuous_update',
 'description',
 'description_tooltip',
 'disabled',
 'layout',
 'max',
 'min',
 'orientation',
 'readout',
 'readout_format',
 'step',
 'style',
 'value']

In [7]:
w.max

100

In [17]:
t = widgets.IntSlider(
    value=7,
    min=0,
    max=10,
    step=1,
    description='testing:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

In [18]:
display(t)
display(w)

IntSlider(value=7, continuous_update=False, description='testing:', max=10)

IntSlider(value=20, description='Temperature: ', max=120, min=20)

In [28]:
print(t.value + w.value)

57


In [35]:
e = widgets.IntRangeSlider(
    value=[5, 7],
    min=0,
    max=10,
    step=1,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

In [37]:
display(e)
display(t)

IntRangeSlider(value=(2, 7), continuous_update=False, description='Test:', max=10)

IntSlider(value=3, continuous_update=False, description='testing:', max=10)

In [38]:
e.value

(2, 7)

In [39]:
widgets.IntProgress(
    value=7,
    min=0,
    max=10,
    step=1,
    description='Loading:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal'
)

IntProgress(value=7, description='Loading:', max=10)

In [52]:
widgets.BoundedIntText(
    value=7,
    min=0,
    max=10,
    step=1,
    description='Text:',
    disabled=False
)

BoundedIntText(value=7, description='Text:', max=10)

In [60]:
b = widgets.Checkbox(
    value=False,
    description='Check me',
    disabled=False,
    indent=False
)

In [61]:
display(b)

Checkbox(value=False, description='Check me', indent=False)

In [62]:
b.value

True

In [71]:
#use checkbox from above to declare validity
widgets.Valid(
    value=b.value,
    description='Valid',
)

Valid(value=False, description='Valid')

In [72]:
widgets.SelectionSlider(
    options=['scrambled', 'sunny side up', 'poached', 'over easy'],
    value='sunny side up',
    description='I like my eggs ...',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)

SelectionSlider(continuous_update=False, description='I like my eggs ...', index=1, options=('scrambled', 'sun…

In [73]:

widgets.ToggleButtons(
    options=['Slow', 'Regular', 'Fast'],
    description='Speed:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
#     icons=['check'] * 3
)

ToggleButtons(description='Speed:', options=('Slow', 'Regular', 'Fast'), tooltips=('Description of slow', 'Des…

In [77]:
d = widgets.DatePicker(
    description='Pick a Date',
    disabled=False
)

In [78]:
display(d)

DatePicker(value=None, description='Pick a Date')

In [80]:
d.value

datetime.date(2020, 1, 15)

In [133]:

accordion = widgets.Accordion(children=[widgets.IntSlider(), widgets.Text("test")], titles=('Slider', 'Text'))
accordion

Accordion(children=(IntSlider(value=0), Text(value='test')))

In [83]:

items = [widgets.Label(str(i)) for i in range(8)]
widgets.GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(3, 100px)"))

GridBox(children=(Label(value='0'), Label(value='1'), Label(value='2'), Label(value='3'), Label(value='4'), La…

In [84]:

items = [widgets.Label(str(i)) for i in range(4)]
widgets.Box(items)

Box(children=(Label(value='0'), Label(value='1'), Label(value='2'), Label(value='3')))

In [94]:

a = widgets.IntSlider(description='a')
a.style.handle_color = 'lightblue'
b = widgets.IntSlider(description='b')
b.style.handle_color = 'green'
c = widgets.IntSlider(description='c')
c.style.handle_color = 'red'
def f(a, b, c):
    print('{}*{}*{}={}'.format(a, b, c, a*b*c))

out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

widgets.HBox([widgets.VBox([a, b, c]), out])


In [88]:

import threading
from IPython.display import display, HTML
import ipywidgets as widgets
import time

def thread_func(something, out):
    for i in range(1, 5):
        time.sleep(1.3)
        out.append_stdout('{} {} {}\n'.format(i, '**'*i, something))
    out.append_display_data(HTML("<em>All done!</em>"))

display('Display in main thread')
out = widgets.Output()
# Now the key: the container is displayed (while empty) in the main thread
display(out)

thread = threading.Thread(
    target=thread_func,
    args=("calculating results", out))
thread.start()

'Display in main thread'

Output()

In [89]:

out.clear_output()

In [92]:
from ipywidgets import Button, HBox, VBox

words = ['correct', 'horse', 'battery', 'staple']
items = [Button(description=w) for w in words]
left_box = VBox([items[0], items[1]])
right_box = VBox([items[2], items[3]])
HBox([left_box, right_box])

In [113]:

from ipywidgets import Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider

form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

form_items = [
    Box([Label(value='Age of the captain'), IntSlider(min=40, max=60)], layout=form_item_layout),
    Box([Label(value='Egg style'),
         Dropdown(options=['Scrambled', 'Sunny side up', 'Over easy'])], layout=form_item_layout),
    Box([Label(value='Ship size'),
         FloatText()], layout=form_item_layout),
    Box([Label(value='Information'),
         Textarea()], layout=form_item_layout)
]

form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    width='65%'
))
form

Box(children=(Box(children=(Label(value='Age of the captain'), IntSlider(value=40, max=60, min=40)), layout=La…

In [110]:
form._view_name

'BoxView'

In [111]:
caption = widgets.Label(value='The values of slider1 and slider2 are synchronized')
sliders1, slider2 = widgets.IntSlider(description='Slider 1'),\
                    widgets.IntSlider(description='Slider 2')
l = widgets.link((sliders1, 'value'), (slider2, 'value'))
display(caption, sliders1, slider2)

Label(value='The values of slider1 and slider2 are synchronized')

IntSlider(value=0, description='Slider 1')

IntSlider(value=0, description='Slider 2')

In [116]:
from ipywidgets import TwoByTwoLayout

In [131]:
app = TwoByTwoLayout(top_left=a, top_right=b,
                     bottom_left=c, bottom_right=d)

app

TwoByTwoLayout(children=(IntSlider(value=41, description='a', layout=Layout(grid_area='top-left'), style=Slide…

In [130]:

interact_manual(slow_function,i=FloatSlider(min=1e5, max=1e7, step=1e5));

NameError: name 'interact_manual' is not defined

## Calling Libraries

In [298]:
%matplotlib inline

import os
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import date
from datetime import datetime
from datetime import timedelta

from darksky import forecast

import config

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import TwoByTwoLayout
from ipywidgets import GridspecLayout

## Loading Models & Data

In [152]:
df_fires = pd.read_csv('data/FireIntensity_Model_June12_Clean.csv')

print(df_fires.shape)

(1328922, 26)


In [159]:
df_fires.longitude.min()

-124.6325

In [200]:
df_fires.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1328922 entries, 0 to 1328921
Data columns (total 26 columns):
Unnamed: 0                 1328922 non-null int64
latitude                   1328922 non-null float64
longitude                  1328922 non-null float64
doy                        1328922 non-null int64
month                      1328922 non-null int64
year                       1328922 non-null int64
cluster_reference          1328922 non-null object
fire_region                1328922 non-null int64
season                     1328922 non-null int64
covertype                  1328922 non-null int64
fuelcode                   1328922 non-null int64
prefire_fuel               1328922 non-null float64
fuel_moisture_class        1328922 non-null int64
temperature                1328922 non-null float64
humidity                   1328922 non-null float64
precip_intensity           1328922 non-null float64
visibility                 1328922 non-null float64
wind_speed            

In [220]:
df_fires.fire_region.value_counts()

0    234640
2    229629
3    224810
1    211145
5    157798
4    100561
7     86791
6     83548
Name: fire_region, dtype: int64

## Defining widgets

In [186]:
datepicker = widgets.DatePicker(
    description='Fire start: ',
    disabled=False,
    value = date.today()
)

In [191]:
temperature = widgets.IntSlider(value=85, max=120, min=20, description='Temperature: ')
temperature.style.handle_color = 'lightblue'

In [ ]:
humidity = widgets.IntSlider(value=85, max=120, min=20, description='Humidity: ')

In [197]:
lat = widgets.FloatSlider(value=df_fires.latitude.median(),
                          max=df_fires.latitude.max(),
                          min=df_fires.latitude.min(),
                          description='Latitude: ',
                          readout_format='.4f',
                          step=0.0001,
                         )
lat.style.handle_color = 'lightpink'

lon = widgets.FloatSlider(value=df_fires.longitude.median(),
                          max=df_fires.longitude.max(),
                          min=df_fires.longitude.min(),
                          description='Longitude: ',
                          readout_format='.4f',
                          step=0.0001,
                         )
lon.style.handle_color = 'lightpink'

In [267]:
fuel_moist = widgets.Dropdown(
    options=[('Very Dry', 1), ('Dry', 2), ('Moderate', 3), ('Moist', 4)],
    value=1,
    description='Fuel Moisture Class:',
)

In [185]:
date.today()

datetime.date(2020, 6, 17)

## Predicting Fire Intensity App

#### Enter the information in app below and submit for pridiction on fire intensity

In [268]:
fireapp = GridspecLayout(4, 3)
fireapp[0,0] = datepicker
fireapp[0,2] = temperature
fireapp[1,0] = lat
fireapp[2,0] = lon
fireapp[0,1] = fuel_moist

fireapp

GridspecLayout(children=(DatePicker(value=datetime.date(2020, 6, 17), description='Fire start: ', layout=Layou…

In [272]:
#source: https://towardsdatascience.com/heres-how-to-calculate-distance-between-2-geolocations-in-python-93ecab5bbba4

# estimate fire region input
def haversine_distance(lat1, lon1, lat2, lon2):
   r = 6371
   phi1 = np.radians(lat1)
   phi2 = np.radians(lat2)
   delta_phi = np.radians(lat2 - lat1)
   delta_lambda = np.radians(lon2 - lon1)
   a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2
   res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
   return np.round(res, 2)

distances_km = []

for row in df_fires.itertuples(index=False):
   distances_km.append(
       haversine_distance(lat.value, lon.value, row.latitude, row.longitude)
   )

df_fires['DistanceFromPoint'] = distances_km

fire_region_in = df_fires[df_fires.DistanceFromPoint == df_fires.DistanceFromPoint.min()].fire_region.min()

In [273]:
#get doy input
def get_season(doy):
    if ((doy >= 80) and (doy <= 172)):
        s = 0  # spring
    elif ((doy > 172) and (doy < 264)):
        s = 1  # summer
    elif ((doy >= 264) and (doy <= 355)):
        s = 2  # fall
    #elif ((doy > 355) and (doy < 80)):
        #s = 3  # winter
    else:
        s = 3
     #   raise IndexError("Invalid date")
    return s

season_in = get_season(datepicker.value.timetuple().tm_yday)

In [305]:
print(lat.value)
print(lon.value)
print(datepicker.value)
print(fire_region_in)
print(season_in)

44.8587
-98.3323
2019-11-13
7
2


In [ ]:
def weather_lookup(date, lat, lon, key=RAPIDAPI_KEY):
        ts = date.isoformat() 
        weather = forecast(key, lat, lon, time=ts)
        w_dict = weather['currently']
        data.append(w_dict)
    return data

In [304]:
dt = datetime(datepicker.value.year, datepicker.value.month, datepicker.value.day, 12).isoformat()

forecast(config.darksky_api1, lat, lon, dt)

HTTPError: Bad response

In [295]:
datepicker.value + timedelta(hours=12)

datetime.date(2019, 11, 13)

In [302]:
datetime(datepicker.value.year, datepicker.value.month, datepicker.value.day, 12).isoformat()

'2019-11-13T12:00:00'

44.8587
-98.3323
2019-11-13
7
2


In [296]:
datepicker.value.year

2019

In [ ]:
# pull from darksky api
'temperature', 'wind_speed','humidity','visibility', 'precip_intensity', 'wind_gust'

#use functions to create
'fire_region', 'season',

#set by user
'fuel_moisture_class'

'fuelcode', 'prefire_fuel'
